# Capstone Project
This is the Coursera Data Science Capstone Project. This is the 9. and last of the 9-Parts Data Science Specialization Path.

<h2>  Data Wrangling</h2>


### 1. Downlaoding and Exploring Data
Firstly we download the necessary  Libraries

In [153]:
!pip install geopy
!pip install geocoder
!pip install folium

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder
import requests # library to handle requests
from bs4 import BeautifulSoup  # library to parse HTML and XML documents

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.metrics import silhouette_score

%matplotlib notebook

print('All libraries imported.')

All libraries imported.


### Data Retrieval
We build the code to scrape the following Wikipedia page; https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M,
in order to obtain the data that we will work on.<p>
List of postal codes of Canada: M

In [211]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(url)
len(df)

3

> #### 0. index of our dataframe is the data, that we will work on and the dataframe is consist of three columns: PostalCode, Borough, and Neighborhood

In [226]:
df1=df[0]
df1.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In our Dataframe there are more than one neighborhood can exist in one postal code area.
For example, we can easily notice that M5A is listed twice and has two neighborhoods:
Harbourfront and Regent Park.

We delete rows where Borough is "Not Assigned"

In [227]:
df1 = df1[df1.Borough!= "Not assigned"]
df1

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [228]:
df1.shape

(103, 3)

### 2. Getting the Coordinates Data

We read csv Coordinate values like Latitude and Longitude from url

In [229]:
dfc=pd.read_csv("http://cocl.us/Geospatial_data")

dfc.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### 3. Merging Coordinates and List of Postal Code Data

Then we merge two dataframe List of postal codes of Canada as df1 and Coordinates as dfc 

In [231]:
dfm = pd.merge(df1, dfc, on="Postal Code")
dfm

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


we are formatting our data frame

In [234]:
format_dfm = {'Latitude':'{:.4f}', 'Longitude':'{:.4f}'}

In [235]:
(dfm.style.format(format_dfm)
 .hide_index()
 .background_gradient(subset='Latitude', cmap='Blues')
 .background_gradient(subset='Longitude', cmap='Reds')
)

Postal Code,Borough,Neighbourhood,Latitude,Longitude
M3A,North York,Parkwoods,43.7533,-79.3297
M4A,North York,Victoria Village,43.7259,-79.3156
M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606
M6A,North York,"Lawrence Manor, Lawrence Heights",43.7185,-79.4648
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895
M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.6679,-79.5322
M1B,Scarborough,"Malvern, Rouge",43.8067,-79.1944
M3B,North York,Don Mills,43.7459,-79.3522
M4B,East York,"Parkview Hill, Woodbine Gardens",43.7064,-79.3099
M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3789


We get the number of Borough values

In [237]:
dfm.Borough.value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
York                 5
East Toronto         5
East York            5
Mississauga          1
Name: Borough, dtype: int64

We get the number of Neighbourhood values

In [238]:
dfm.Neighbourhood.value_counts()

Downsview                                                        4
Don Mills                                                        2
Willowdale, Willowdale West                                      1
Davisville North                                                 1
Parkdale, Roncesvalles                                           1
                                                                ..
Toronto Dominion Centre, Design Exchange                         1
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood    1
Roselawn                                                         1
Stn A PO Boxes                                                   1
Fairview, Henry Farm, Oriole                                     1
Name: Neighbourhood, Length: 99, dtype: int64

### 4. Getting Latitude and Longitude 

##### Use geopy library to get the latitude and longitude values of Toronto
In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In [240]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


we visualize  the data

In [242]:
map_toronto = folium.Map(location=[43.653963, -79.387207], zoom_start=10)

# add markers to map
for lat, lng, Borough, Neighbourhood in zip(dfm['Latitude'], dfm['Longitude'], dfm['Borough'], dfm['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Toronto. So let's slice the original dataframe and create a new dataframe of the Toronto data.

### 5. Defining Foursquare Credentials and Version

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

In [248]:
CLIENT_ID = '1MYGIJZYDXLLUZUQPN3KGEGEO21HDH1KRMZCEXSPAPNH3CFD' # your Foursquare ID
CLIENT_SECRET = 'NXSCQIKORXTPTNKJ0SRBW5M4PADCOUYYXOBTYC0CBPVRZYJW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1MYGIJZYDXLLUZUQPN3KGEGEO21HDH1KRMZCEXSPAPNH3CFD
CLIENT_SECRET:NXSCQIKORXTPTNKJ0SRBW5M4PADCOUYYXOBTYC0CBPVRZYJW


#### Let's explore the first neighborhood in our dataframe.
Get the neighborhood's name.

In [249]:
dfm.loc[0, 'Neighbourhood']

'Parkwoods'

In [250]:
neighbourhood_latitude = dfm.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = dfm.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = dfm.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [251]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=1MYGIJZYDXLLUZUQPN3KGEGEO21HDH1KRMZCEXSPAPNH3CFD&client_secret=NXSCQIKORXTPTNKJ0SRBW5M4PADCOUYYXOBTYC0CBPVRZYJW&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=100'

#### Send the GET request and examine the resutls

In [252]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ffa276f720fd131772b05e9'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': 

### Function that extracts the category of the venue

In [254]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a _pandas_ dataframe.

In [255]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng', 'venue.location.city']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng,city
0,Brookbanks Park,Park,43.751976,-79.332140,Toronto
1,Variety Store,Food & Drink Shop,43.751974,-79.333114,Toronto


In [256]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


### 5. Creating a function to repeat the same process to all the neighborhoods in Toronto

In [257]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [258]:
northyork_venues = getNearbyVenues(names=northyork_data['Neighbourhood'],
                                   latitudes=northyork_data['Latitude'],
                                   longitudes=northyork_data['Longitude']
                                  )

Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Bedford Park, Lawrence Manor East
Lawrence Manor, Lawrence Heights
Glencairn
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Humberlea, Emery


In [259]:
dfm_venues = getNearbyVenues(names=dfm['Neighbourhood'],
                                   latitudes=northyork_data['Latitude'],
                                   longitudes=northyork_data['Longitude'])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside


In [260]:
print(dfm_venues.shape)
dfm_venues.head()

(244, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.803762,-79.363452,New York Fries,43.803664,-79.363905,Fast Food Restaurant
1,Parkwoods,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
2,Parkwoods,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
3,Parkwoods,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
4,Parkwoods,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run


Let's check how many venues were returned for each neighborhood

In [261]:
dfm_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,4,4,4,4,4,4
Caledonia-Fairbanks,3,3,3,3,3,3
Don Mills,9,9,9,9,9,9
"Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood",5,5,5,5,5,5
"Garden District, Ryerson",5,5,5,5,5,5
Glencairn,22,22,22,22,22,22
"Guildwood, Morningside, West Hill",21,21,21,21,21,21
Humewood-Cedarvale,4,4,4,4,4,4
"Islington Avenue, Humber Valley Village",35,35,35,35,35,35


#### Let's find out how many unique categories can be curated from all the returned venues

In [262]:
print('There are {} uniques categories.'.format(len(dfm_venues['Venue Category'].unique())))

There are 100 uniques categories.


### 6. Analyze Each Neighborhood

In [263]:
# one hot encoding
dfm_onehot = pd.get_dummies(dfm_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dfm_onehot['Neighbourhood'] = dfm_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [dfm_onehot.columns[-1]] + list(dfm_onehot.columns[:-1])
dfm_onehot = dfm_onehot[fixed_columns]

dfm_onehot.head()

,Neighbourhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [264]:
dfm_onehot.shape

(244, 101)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [265]:
toronto_grouped = dfm_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.00,0.250000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Caledonia-Fairbanks,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.333333,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Don Mills,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"Garden District, Ryerson",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.20,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Glencairn,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.045455,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,"Guildwood, Morningside, West Hill",0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.000000,0.047619,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000
7,Humewood-Cedarvale,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,"Islington Avenue, Humber Valley Village",0.000000,0.000000,0.000000,0.000000,0.028571,0.000000,0.00,0.000000,0.028571,...,0.028571,0.000000,0.057143,0.000000,0.000000,0.000000,0.000000,0.000000,0.028571,0.000000
9,"Lawrence Manor, Lawrence Heights",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


 Let's confirm the new size


In [266]:
toronto_grouped.shape

(23, 101)

#### Let's print each neighborhood along with the top 5 most common venues


In [267]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                 venue  freq
0     Asian Restaurant  0.25
1               Bakery  0.25
2                  Pub  0.25
3  Japanese Restaurant  0.25
4    Accessories Store  0.00


----Caledonia-Fairbanks----
                        venue  freq
0  Construction & Landscaping  0.33
1                      Bakery  0.33
2                        Park  0.33
3           Accessories Store  0.00
4                Liquor Store  0.00


----Don Mills----
            venue  freq
0   Grocery Store  0.11
1  Discount Store  0.11
2     Coffee Shop  0.11
3        Pharmacy  0.11
4     Pizza Place  0.11


----Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood----
                   venue  freq
0            Coffee Shop   0.2
1           Hockey Arena   0.2
2      French Restaurant   0.2
3           Intersection   0.2
4  Portuguese Restaurant   0.2


----Garden District, Ryerson----
                  venue  freq
0                   Gym   0.2
1  Caribbean Restaurant   0.2
2            

#### Let's put that into a _pandas_ dataframe
First, let's write a function to sort the venues in descending order.

In [268]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.


In [269]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Japanese Restaurant,Asian Restaurant,Bakery,Pub,Women's Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
1,Caledonia-Fairbanks,Park,Construction & Landscaping,Bakery,Women's Store,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Convenience Store,Cosmetics Shop
2,Don Mills,Airport,Coffee Shop,Butcher,Grocery Store,Park,Business Service,Pharmacy,Pizza Place,Discount Store,Women's Store
3,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",French Restaurant,Coffee Shop,Hockey Arena,Portuguese Restaurant,Intersection,Women's Store,Dim Sum Restaurant,Clothing Store,Comfort Food Restaurant,Construction & Landscaping
4,"Garden District, Ryerson",Caribbean Restaurant,Café,Gym,Athletics & Sports,Japanese Restaurant,Women's Store,Discount Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store


### 7. Clustering Neighborhoods


Run _k_-means to cluster the neighborhood into 5 clusters.


In [270]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 2, 1, 1, 1, 1, 1, 2, 1, 3], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [271]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = dfm

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Golf Course,Mediterranean Restaurant,Fast Food Restaurant,Pool,Dog Run,Women's Store,Department Store,Clothing Store,Coffee Shop,Comfort Food Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Japanese Restaurant,Restaurant,Women's Store,Jewelry Store,Juice Bar,Shoe Store,Bakery
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,Chinese Restaurant,Bank,Café,Japanese Restaurant,Food Truck,Food Court,Coffee Shop,Fried Chicken Joint,Comfort Food Restaurant,Construction & Landscaping
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,3.0,Martial Arts School,Women's Store,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,4.0,Park,Diner,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega


### 8. Examine Clusters


Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.


#### Cluster 1

In [272]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Downtown Toronto,0.0,Food Truck,Business Service,Baseball Field,Women's Store,Discount Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
23,East York,0.0,Paper / Office Supplies Store,Baseball Field,Discount Store,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega


#### Cluster 2

In [273]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1.0,Golf Course,Mediterranean Restaurant,Fast Food Restaurant,Pool,Dog Run,Women's Store,Department Store,Clothing Store,Coffee Shop,Comfort Food Restaurant
1,North York,1.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Japanese Restaurant,Restaurant,Women's Store,Jewelry Store,Juice Bar,Shoe Store,Bakery
2,Downtown Toronto,1.0,Chinese Restaurant,Bank,Café,Japanese Restaurant,Food Truck,Food Court,Coffee Shop,Fried Chicken Joint,Comfort Food Restaurant,Construction & Landscaping
5,Etobicoke,1.0,Ramen Restaurant,Sandwich Place,Coffee Shop,Sushi Restaurant,Restaurant,Pizza Place,Shopping Mall,Café,Middle Eastern Restaurant,Bubble Tea Shop
7,North York,1.0,Airport,Coffee Shop,Butcher,Grocery Store,Park,Business Service,Pharmacy,Pizza Place,Discount Store,Women's Store
9,Downtown Toronto,1.0,Caribbean Restaurant,Café,Gym,Athletics & Sports,Japanese Restaurant,Women's Store,Discount Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
10,North York,1.0,Coffee Shop,Beer Store,Restaurant,Gym,Discount Store,Sandwich Place,Japanese Restaurant,Bus Line,Italian Restaurant,Dim Sum Restaurant
11,Etobicoke,1.0,Coffee Shop,Bank,Gas Station,Sandwich Place,Middle Eastern Restaurant,Intersection,Ice Cream Shop,Pharmacy,Pizza Place,Bridal Shop
12,Scarborough,1.0,Furniture / Home Store,Bar,Coffee Shop,Caribbean Restaurant,Massage Studio,Miscellaneous Shop,Vietnamese Restaurant,Food Court,Department Store,Fried Chicken Joint
13,North York,1.0,Airport,Coffee Shop,Butcher,Grocery Store,Park,Business Service,Pharmacy,Pizza Place,Discount Store,Women's Store


#### Cluster 3

In [274]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,East York,2.0,Grocery Store,Park,Bank,Shopping Mall,Women's Store,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
16,York,2.0,Discount Store,Grocery Store,Gym / Fitness Center,Athletics & Sports,Women's Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
21,York,2.0,Park,Construction & Landscaping,Bakery,Women's Store,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Convenience Store,Cosmetics Shop


#### Cluster 4

In [275]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,North York,3.0,Martial Arts School,Women's Store,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega


#### Cluster 5

In [276]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Downtown Toronto,4.0,Park,Diner,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
6,Scarborough,4.0,Park,Convenience Store,Diner,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega
8,East York,4.0,Park,Food & Drink Shop,Diner,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
